In [1]:
import torch
import cv2
import numpy
import torch.nn as nn
import torch.nn.functional as F

# Dymically generate residual blocks 

In [ ]:
class ResBlock(nn.Module):
    def __init__(self, n_channels):
        super(ResBlock, self).__init__()
        self.conv = nn.Conv2d(n_channels, n_channels, kernel_size=3, padding=1, bias=False)
        self.batch_norm = nn.BatchNorm2d(num_features= n_channels)
        torch.nn.init.kaiming_normal_(self.conv.weight, nonlinearity='relu')
        torch.nn.init.constant_(self.batch_norm.weight, 0.5)
        torch.nn.init.zeros_(self.batch_norm.bias)
        
    def forward(self, x):
        out = self.conv(x)
        out = self.batch_norm(out)
        out = torch.relu(out) 
#         print(out.shape)
        return out + x
    
class NetResDeep(nn.Module):
    def __init__(self, n_channel1=32, n_blocks=10):
        super().__init__()
        self.nchannel1 = n_channel1
        self.Conv1 = nn.Conv2d(3, n_channel1, kernel_size=3, padding=1)
        self.resblocks = nn.Sequential(*(n_blocks*[ResBlock(self.nchannel1)]))
        self.fc1 = nn.Linear(n_channel1*60*60, 32)
        self.fc2 = nn.Linear(32,2)
        
    def forward(self, x):
        out = F.max_pool2d(torch.relu(self.Conv1(x)), 2)
        out = self.resblocks(out)
        out = F.max_pool2d(out, 2)
        out = out.view(-1, 60 * 60 * self.nchannel1)
        out = torch.relu(self.fc1(out))
        out = self.fc2(out)
        
        return out

# call block

In [ ]:
model = NetResDeep()
output = model(img.unsqueeze(0))